In this notebook I'll be refining what I started yesterday. I will be looking for a better way to visualize total daily insulin along with daily CGM stats.

In [2]:
# IMPORTS

# since this notebook lives in a sub-folder of the main project, I'll add the main folder to the python path
import sys
sys.path.append("../")

# Load my module
from tools.glooko import read_all

# Load extra stuff that I need
import numpy as np

# Plotting (matplotlib widge allows for interactivity in the live notebook)
import matplotlib.pyplot as plt
%matplotlib widget


In [ ]:
# LOAD DATA
# Note that I added the carb and insulin correction columns to the read_all module 
df_cgm, df_bolus, df_basal, df_insulin = read_all(r"../data")

# This is important later when doing groupby
df_cgm.drop("sn_cgm", axis=1, inplace=True)